## 1. Prepare Environments

* 데이터 로드를 위한 구글 드라이브를 마운트합니다.
* 필요한 라이브러리를 설치합니다.

In [ ]:
# # 필요한 라이브러리를 설치합니다.
# !pip install timm
# !pip install matplotlib
# !pip install seaborn
# !pip install optuna

## 2. Import Library & Define Functions
* 학습 및 추론에 필요한 라이브러리를 로드합니다.
* 학습 및 추론에 필요한 함수와 클래스를 정의합니다.

In [ ]:
import os
import time
import random
import copy

import optuna, math
import timm
import torch
import albumentations as A
import pandas as pd
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
from albumentations.pytorch import ToTensorV2
from torch.optim import Adam, AdamW  # AdamW 추가
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import CosineAnnealingLR, OneCycleLR  # OneCycleLR 추가
from torch.cuda.amp import autocast, GradScaler  # Mixed Precision용

from PIL import Image
from tqdm import tqdm
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report
from sklearn.model_selection import train_test_split, StratifiedKFold

from collections import Counter
import warnings
warnings.filterwarnings('ignore')

def mixup_data(x, y, alpha=1.0):
    if alpha > 0:
        lam = np.random.beta(alpha, alpha)
    else:
        lam = 1
    batch_size = x.size()[0]
    index = torch.randperm(batch_size).cuda()
    mixed_x = lam * x + (1 - lam) * x[index, :]
    y_a, y_b = y, y[index]
    return mixed_x, y_a, y_b, lam



# WandB 관련 import 추가
import wandb
from datetime import datetime

In [ ]:
# =============================================================================
# 1-1. WandB Login and Configuration
# =============================================================================
"""
🚀 팀원 사용 가이드:

1. WandB 계정 생성: https://wandb.ai/signup
2. 이 셀 실행 시 로그인 프롬프트가 나타나면 개인 API 키 입력
3. EXPERIMENT_NAME을 다음과 같이 변경:
   - "member1-baseline"
   - "member2-augmentation-test"  
   - "member3-hyperparameter-tuning"
   등등 각자 다른 이름 사용

4. 팀 대시보드 URL: [여기에 당신의 프로젝트 URL 추가]

⚠️ 주의사항:
- 절대 API 키를 코드에 하드코딩하지 마세요
- EXPERIMENT_NAME만 변경하고 PROJECT_NAME은 그대로 두세요
- 각자 개인 계정으로 로그인해서 실험을 추가하세요
"""

# WandB 로그인 (각자 실행)
try:
    if wandb.api.api_key is None:
        print("WandB에 로그인이 필요합니다.")
        wandb.login()
    else:
        print(f"WandB 로그인 상태: {wandb.api.viewer()['username']}")
except:
    print("WandB 로그인을 진행합니다...")
    wandb.login()

# 프로젝트 설정 (각자 수정할 부분)
PROJECT_NAME = "document-classification-team"  # 모든 팀원 동일
ENTITY = None  # 각자 개인 계정 사용
EXPERIMENT_NAME = "swinb-convnext-ensemble"  # 팀원별로 변경 (예: "member1-hyperopt", "member2-augmentation")

print(f"프로젝트: {PROJECT_NAME}")
print(f"실험명: {EXPERIMENT_NAME}")
print("팀원들은 EXPERIMENT_NAME을 각자 다르게 변경해주세요!")

In [ ]:
# 시드를 고정합니다.
SEED = 42
os.environ['PYTHONHASHSEED'] = str(SEED)
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.benchmark = True

In [ ]:
# Device 및 기본 설정

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"📱 Using device: {device}")

data_path = '/root/home/cv_contest/CV_data/train'

In [ ]:
# 다중 모델 설정

MODEL_CONFIGS = {
    'swin_b': {
        'model_name': 'swin_base_patch4_window12_384_in22k',
        'img_size': 384,
        'batch_size': 16,
        'lr': 5e-4,
        'architecture': 'Swin Transformer',
        'weight_decay': 0.01
    },
    'convnext': {
        'model_name': 'convnext_base_in22k',
        'img_size': 224,
        'batch_size': 16,
        'lr': 3e-4,
        'architecture': 'ConvNeXt',
        'weight_decay': 0.01
    }
}

# 사용할 모델들 선택
ACTIVE_MODELS = ['swin_b', 'convnext']
EPOCHS = 50
num_workers = 30
N_FOLDS = 5

print(f"📊 사용할 모델: {ACTIVE_MODELS}")
for model_key in ACTIVE_MODELS:
    config = MODEL_CONFIGS[model_key]
    print(f"  🤖 {model_key}: {config['model_name']} ({config['img_size']}x{config['img_size']})")


In [ ]:
# Multi-Resolution Dataset 클래스

class MultiResImageDataset(Dataset):
    def __init__(self, data, path, img_sizes, epoch=0, total_epochs=10, is_train=True):
        """
        다중 해상도를 지원하는 Dataset
        """
        if isinstance(data, str):
            self.df = pd.read_csv(data).values
        else:
            self.df = data.values
        self.path = path
        self.img_sizes = img_sizes
        self.epoch = epoch
        self.total_epochs = total_epochs
        self.is_train = is_train
        
        # Hard augmentation 확률 계산
        self.p_hard = 0.2 + 0.3 * (epoch / total_epochs) if is_train else 0
        
        # 각 해상도별 transform 생성
        self.transforms = {}
        for model_key, img_size in img_sizes.items():
            # Normal augmentation
            normal_aug = A.Compose([
                A.LongestMaxSize(max_size=img_size),
                A.PadIfNeeded(min_height=img_size, min_width=img_size, border_mode=0, value=0),
                A.OneOf([
                    A.Rotate(limit=[90,90], p=1.0),
                    A.Rotate(limit=[180,180], p=1.0),
                    A.Rotate(limit=[270,270], p=1.0),
                ], p=0.6),
                A.RandomBrightnessContrast(brightness_limit=0.3, contrast_limit=0.3, p=0.8),
                A.GaussNoise(var_limit=(30.0, 100.0), p=0.7),
                A.HorizontalFlip(p=0.5),
                A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
                ToTensorV2(),
            ])
            
            # Hard augmentation
            hard_aug = A.Compose([
                A.LongestMaxSize(max_size=img_size),
                A.PadIfNeeded(min_height=img_size, min_width=img_size, border_mode=0, value=0),
                A.OneOf([
                    A.Rotate(limit=[90,90], p=1.0),
                    A.Rotate(limit=[180,180], p=1.0),
                    A.Rotate(limit=[270,270], p=1.0),
                    A.Rotate(limit=[-15,15], p=1.0),
                ], p=0.8),
                A.OneOf([
                    A.MotionBlur(blur_limit=15, p=1.0),
                    A.GaussianBlur(blur_limit=15, p=1.0),
                ], p=0.95),
                A.RandomBrightnessContrast(brightness_limit=0.5, contrast_limit=0.5, p=0.9),
                A.GaussNoise(var_limit=(50.0, 150.0), p=0.8),
                A.JpegCompression(quality_lower=70, quality_upper=100, p=0.5),
                A.HorizontalFlip(p=0.5),
                A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
                ToTensorV2(),
            ])
            
            self.transforms[model_key] = {
                'normal': normal_aug,
                'hard': hard_aug
            }

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        name, target = self.df[idx]
        img = np.array(Image.open(os.path.join(self.path, name)))
        
        # 각 모델별로 다른 해상도의 이미지 생성
        images = {}
        for model_key in self.img_sizes.keys():
            # 배치별 증강 선택
            if self.is_train and random.random() < self.p_hard:
                aug_img = self.transforms[model_key]['hard'](image=img)['image']
            else:
                aug_img = self.transforms[model_key]['normal'](image=img)['image']
            images[model_key] = aug_img
        
        return images, target

In [ ]:
# 다중 모델 학습 함수들

def train_multi_models_epoch(loaders, models, optimizers, loss_fns, device, epoch=None, fold=None):
    """다중 모델을 동시에 학습하는 함수"""
    scaler = GradScaler()
    
    # 모든 모델을 학습 모드로
    for model in models.values():
        model.train()
    
    model_losses = {key: 0 for key in models.keys()}
    model_preds = {key: [] for key in models.keys()}
    targets_list = []
    
    # 첫 번째 loader 기준으로 iteration
    first_key = list(loaders.keys())[0]
    pbar = tqdm(loaders[first_key], desc=f"Training Epoch {epoch+1 if epoch else '?'}")
    
    for batch_idx, batch_data in enumerate(pbar):
        images_dict, targets = batch_data
        targets = targets.to(device)
        
        # 각 모델별로 학습
        batch_losses = {}
        for model_key in models.keys():
            model = models[model_key]
            optimizer = optimizers[model_key]
            loss_fn = loss_fns[model_key]
            
            image = images_dict[model_key].to(device)
            
            # Mixup 확률 (30% 유지)
            mixup_applied = False
            if random.random() < 0.3:
                mixed_x, y_a, y_b, lam = mixup_data(image, targets, alpha=1.0)
                with autocast():
                    preds = model(mixed_x)
                loss = lam * loss_fn(preds.float(), y_a) + (1 - lam) * loss_fn(preds.float(), y_b)
                mixup_applied = True
            else:
                with autocast():
                    preds = model(image)
                loss = loss_fn(preds.float(), targets)
            
            # 역전파
            optimizer.zero_grad(set_to_none=True)
            scaler.scale(loss).backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            scaler.step(optimizer)
            scaler.update()
            
            batch_losses[model_key] = loss.item()
            model_losses[model_key] += loss.item()
            model_preds[model_key].extend(preds.argmax(dim=1).detach().cpu().numpy())
        
        targets_list.extend(targets.detach().cpu().numpy())
        
        # 배치별 로깅
        if batch_idx % 100 == 0 and wandb.run is not None:
            step = epoch * len(pbar) + batch_idx if epoch is not None else batch_idx
            log_data = {f"fold_{fold}/batch_step": step}
            for model_key, loss_val in batch_losses.items():
                log_data[f"fold_{fold}/{model_key}_batch_loss"] = loss_val
            log_data[f"fold_{fold}/mixup_applied"] = int(mixup_applied)
            wandb.log(log_data)
        
        pbar.set_description(f"Losses: {', '.join([f'{k}: {v:.4f}' for k, v in batch_losses.items()])}")
    
    # 각 모델별 결과 계산
    results = {}
    for model_key in models.keys():
        model_losses[model_key] /= len(pbar)
        train_acc = accuracy_score(targets_list, model_preds[model_key])
        train_f1 = f1_score(targets_list, model_preds[model_key], average='macro')
        
        results[model_key] = {
            "train_loss": model_losses[model_key],
            "train_acc": train_acc,
            "train_f1": train_f1,
        }
    
    return results

def validate_multi_models_epoch(loaders, models, loss_fns, device, epoch=None, fold=None, log_confusion=False):
    """다중 모델을 동시에 검증하는 함수"""
    
    # 모든 모델을 평가 모드로
    for model in models.values():
        model.eval()
    
    model_losses = {key: 0 for key in models.keys()}
    model_preds = {key: [] for key in models.keys()}
    targets_list = []
    
    with torch.no_grad():
        first_key = list(loaders.keys())[0]
        pbar = tqdm(loaders[first_key], desc=f"Validating Epoch {epoch+1 if epoch else '?'}")
        
        for batch_data in pbar:
            images_dict, targets = batch_data
            targets = targets.to(device)
            
            batch_losses = {}
            for model_key in models.keys():
                model = models[model_key]
                loss_fn = loss_fns[model_key]
                
                image = images_dict[model_key].to(device)
                preds = model(image)
                loss = loss_fn(preds, targets)
                
                batch_losses[model_key] = loss.item()
                model_losses[model_key] += loss.item()
                model_preds[model_key].extend(preds.argmax(dim=1).detach().cpu().numpy())
            
            targets_list.extend(targets.detach().cpu().numpy())
            pbar.set_description(f"Val Losses: {', '.join([f'{k}: {v:.4f}' for k, v in batch_losses.items()])}")
    
    # 각 모델별 결과 계산
    results = {}
    for model_key in models.keys():
        model_losses[model_key] /= len(pbar)
        val_acc = accuracy_score(targets_list, model_preds[model_key])
        val_f1 = f1_score(targets_list, model_preds[model_key], average='macro')
        
        # Confusion Matrix 로깅 (마지막 epoch에만)
        if log_confusion and wandb.run is not None:
            try:
                wandb.log({
                    f"fold_{fold}/{model_key}_confusion_matrix": wandb.plot.confusion_matrix(
                        probs=None,
                        y_true=targets_list,
                        preds=model_preds[model_key],
                        class_names=[f"Class_{i}" for i in range(17)]
                    )
                })
            except Exception as e:
                print(f"⚠️ {model_key} Confusion matrix 로깅 실패: {e}")
        
        results[model_key] = {
            "val_loss": model_losses[model_key],
            "val_acc": val_acc,
            "val_f1": val_f1,
        }
    
    return results


In [ ]:
# WandB Config 설정

config = {
    # Multi-Model config
    "ensemble_models": ACTIVE_MODELS,
    "model_configs": MODEL_CONFIGS,
    "multi_resolution": True,
    
    # Training config
    "epochs": EPOCHS,
    "num_workers": num_workers,
    "device": str(device),
    
    # K-Fold config
    "n_folds": N_FOLDS,
    "seed": SEED,
    "cv_strategy": "StratifiedKFold",
    
    # Augmentation & Training techniques
    "mixup_alpha": 1.0,
    "mixup_prob": 0.3,
    "label_smoothing": 0.1,
    "gradient_clipping": 1.0,
    "mixed_precision": True,
    
    # Data
    "data_path": data_path,
    "train_transforms": "Progressive_Hard_Augmentation",
    "test_transforms": "Basic",
}

print("✅ 하이퍼파라미터 설정 완료!")
for model_key in ACTIVE_MODELS:
    config_detail = MODEL_CONFIGS[model_key]
    print(f"🤖 {model_key}: {config_detail['model_name']} ({config_detail['img_size']}px, LR={config_detail['lr']})")


In [ ]:
# 데이터 로드 및 클래스 가중치 계산

skf = StratifiedKFold(n_splits=N_FOLDS, shuffle=True, random_state=42)
train_df = pd.read_csv("CV_data/train.csv")
print(f"📊 학습 데이터: {len(train_df)}개 샘플")

# 클래스 분포 확인
class_counts = train_df['target'].value_counts().sort_index()
print(f"📊 클래스 분포: {dict(class_counts)}")

# 클래스 가중치 계산
total_samples = len(train_df)
num_classes = 17
class_weights = torch.tensor([
    total_samples / (num_classes * class_counts[i]) for i in range(num_classes)
], dtype=torch.float32).to(device)

# 가중치 정규화
class_weights = class_weights / class_weights.mean()
print(f"📊 정규화된 클래스 가중치 범위: {class_weights.min():.3f} ~ {class_weights.max():.3f}")


In [ ]:
# K-Fold 결과 저장용 변수

all_fold_results = {model_key: [] for model_key in ACTIVE_MODELS}
all_fold_models = {model_key: [] for model_key in ACTIVE_MODELS}

# 현재 사용할 이미지 크기들
img_sizes = {model_key: MODEL_CONFIGS[model_key]['img_size'] for model_key in ACTIVE_MODELS}
print(f"📊 이미지 크기 설정: {img_sizes}")

In [ ]:
# WandB 메인 실험 시작

main_run = wandb.init(
    project=PROJECT_NAME,
    entity=ENTITY,
    name=f"{EXPERIMENT_NAME}",
    config=config,
    tags=["multi-model", "swin-convnext", "k-fold-cv", "ensemble", "main-experiment"],
    group="multi-model-experiment",
    job_type="cross-validation",
    notes=f"Swin-B + ConvNeXt with {N_FOLDS}-Fold Cross Validation"
)

print(f"🚀 WandB 실험 시작!")
print(f"📊 대시보드: {main_run.url}")
print(f"📋 실험명: {main_run.name}")

# 데이터셋 정보 로깅
wandb.log({
    "dataset/total_samples": len(train_df),
    "dataset/num_classes": 17,
    "dataset/samples_per_fold": len(train_df) // N_FOLDS,
})

# 클래스 분포 시각화
class_dist_data = [[f"Class_{i}", count] for i, count in enumerate(class_counts)]
wandb.log({
    "dataset/class_distribution": wandb.plot.bar(
        wandb.Table(data=class_dist_data, columns=["Class", "Count"]),
        "Class", "Count", 
        title="Training Data Class Distribution"
    )
})

print(f"\n{'='*60}")
print(f"🎯 {N_FOLDS}-FOLD MULTI-MODEL CROSS VALIDATION 시작")
print(f"{'='*60}")


## K-Fold Cross Validation

In [ ]:
# K-Fold Cross Validation Loop


for fold, (train_idx, val_idx) in enumerate(skf.split(train_df, train_df['target'])):
    print(f"\n{'='*50}")
    print(f"📁 FOLD {fold + 1}/{N_FOLDS} - Multi-Model Training")
    print(f"{'='*50}")
    
    # 각 fold별 child run 생성
    fold_run = wandb.init(
        project=PROJECT_NAME,
        entity=ENTITY,
        name=f"fold-{fold+1}-multi-{datetime.now().strftime('%H%M')}",
        config=config,
        tags=["fold", f"fold-{fold+1}", "multi-model", "child-run"],
        group="multi-model-experiment",
        job_type=f"fold-{fold+1}",
        reinit=True
    )
    
    print(f"📊 Fold {fold+1} Dashboard: {fold_run.url}")
    
    # 데이터 분할
    train_fold_df = train_df.iloc[train_idx].reset_index(drop=True)
    val_fold_df = train_df.iloc[val_idx].reset_index(drop=True)
    
    # Fold별 클래스 분포 확인
    train_class_dist = train_fold_df['target'].value_counts().sort_index()
    val_class_dist = val_fold_df['target'].value_counts().sort_index()
    
    print(f"📊 Fold {fold+1} 클래스 분포:")
    print(f"  Train: {len(train_fold_df)}개 샘플")
    print(f"  Val: {len(val_fold_df)}개 샘플")
    
    wandb.log({
        "fold_info/fold_number": fold + 1,
        "fold_info/train_samples": len(train_fold_df),
        "fold_info/val_samples": len(val_fold_df),
        "fold_info/train_ratio": len(train_fold_df) / len(train_df),
        "fold_info/val_ratio": len(val_fold_df) / len(train_df),
    })
    
    # 다중 해상도 Dataset 생성
    trn_dataset = MultiResImageDataset(
        train_fold_df,
        "/root/home/cv_contest/CV_data/train",
        img_sizes,
        epoch=0,
        total_epochs=EPOCHS,
        is_train=True
    )
    
    val_dataset = MultiResImageDataset(
        val_fold_df,
        "/root/home/cv_contest/CV_data/train",
        img_sizes,
        epoch=0,
        total_epochs=EPOCHS,
        is_train=False
    )
    
    # 각 모델별 DataLoader 생성
    trn_loaders = {}
    val_loaders = {}
    
    for model_key in ACTIVE_MODELS:
        batch_size = MODEL_CONFIGS[model_key]['batch_size']
        
        trn_loaders[model_key] = DataLoader(
            trn_dataset,
            batch_size=batch_size,
            shuffle=True,
            num_workers=num_workers,
            pin_memory=True,
            drop_last=False
        )
        
        val_loaders[model_key] = DataLoader(
            val_dataset,
            batch_size=batch_size,
            shuffle=False,
            num_workers=num_workers,
            pin_memory=True
        )
    
    # 각 모델 생성
    models = {}
    optimizers = {}
    schedulers = {}
    loss_fns = {}
    best_models = {model_key: None for model_key in ACTIVE_MODELS}
    best_f1s = {model_key: 0.0 for model_key in ACTIVE_MODELS}
    
    for model_key in ACTIVE_MODELS:
        config_detail = MODEL_CONFIGS[model_key]
        
        # 모델 생성
        models[model_key] = timm.create_model(
            config_detail['model_name'],
            pretrained=True,
            num_classes=17
        ).to(device)
        
        # 옵티마이저 생성
        optimizers[model_key] = AdamW(
            models[model_key].parameters(),
            lr=config_detail['lr'],
            weight_decay=config_detail['weight_decay']
        )
        
        # 스케줄러 생성
        schedulers[model_key] = CosineAnnealingLR(optimizers[model_key], T_max=EPOCHS)
        
        # 손실 함수 생성 (클래스 가중치 적용)
        loss_fns[model_key] = nn.CrossEntropyLoss(
            weight=class_weights,
            label_smoothing=0.1
        )
        
        model_params = sum(p.numel() for p in models[model_key].parameters())
        print(f"🤖 {model_key} 모델 생성: {config_detail['model_name']} ({model_params:,} parameters)")
    
    # Training Loop for Current Fold
    #patience = {model_key: 0 for model_key in ACTIVE_MODELS}
    #max_patience = 3
    
    print(f"🎯 Fold {fold+1} 다중 모델 학습 시작")
    
    for epoch in range(EPOCHS):
        print(f"\n📈 Epoch {epoch+1}/{EPOCHS}")
        start_time = time.time()
        
        # 에폭별 Dataset 업데이트 (점진적 증강)
        trn_dataset.epoch = epoch
        val_dataset.epoch = epoch
        
        # 모든 모델 동시 학습
        train_results = train_multi_models_epoch(
            trn_loaders, models, optimizers, loss_fns, device,
            epoch=epoch, fold=fold+1
        )
        
        # 모든 모델 동시 검증
        val_results = validate_multi_models_epoch(
            val_loaders, models, loss_fns, device,
            epoch=epoch, fold=fold+1,
            log_confusion=(epoch == EPOCHS-1)
        )
        
        # 각 모델별 스케줄러 업데이트
        for scheduler in schedulers.values():
            scheduler.step()
        
        epoch_time = time.time() - start_time
        
        # WandB 로깅 및 최고 모델 저장
        log_data = {"epoch": epoch + 1, "fold": fold + 1, "epoch_time": epoch_time}
        
        print(f"📊 Epoch {epoch+1:2d} 결과:")
        for model_key in ACTIVE_MODELS:
            train_res = train_results[model_key]
            val_res = val_results[model_key]
            current_lr = optimizers[model_key].param_groups[0]['lr']
            
            log_data.update({
                f"{model_key}/train_loss": train_res['train_loss'],
                f"{model_key}/train_acc": train_res['train_acc'],
                f"{model_key}/train_f1": train_res['train_f1'],
                f"{model_key}/val_loss": val_res['val_loss'],
                f"{model_key}/val_acc": val_res['val_acc'],
                f"{model_key}/val_f1": val_res['val_f1'],
                f"{model_key}/lr": current_lr,
            })
            
            print(f"  {model_key:10s}: Train F1={train_res['train_f1']:.4f}, Val F1={val_res['val_f1']:.4f}, LR={current_lr:.2e}")
            
            # 최고 성능 모델 저장
            if val_res['val_f1'] > best_f1s[model_key]:
                best_f1s[model_key] = val_res['val_f1']
                best_models[model_key] = copy.deepcopy(models[model_key].state_dict())
                #patience[model_key] = 0
                
                # 모델 저장
                model_path = f'best_{model_key}_fold_{fold+1}.pth'
                torch.save(best_models[model_key], model_path)
                wandb.save(model_path, policy="now")
                
                # 새로운 최고 성능 로깅
                wandb.log({
                    f"best_performance/{model_key}_epoch": epoch + 1,
                    f"best_performance/{model_key}_val_f1": best_f1s[model_key],
                    f"best_performance/{model_key}_val_acc": val_res['val_acc'],
                })
                
                print(f"    🎉 {model_key} 새로운 최고 성능! F1: {best_f1s[model_key]:.4f}")
            #else:
                #patience[model_key] += 1
        
        # GPU 메모리 사용량 로깅
        if torch.cuda.is_available():
            gpu_memory_used = torch.cuda.memory_allocated(0) / 1e9
            gpu_memory_total = torch.cuda.get_device_properties(0).total_memory / 1e9
            log_data.update({
                "system/gpu_memory_used_gb": gpu_memory_used,
                "system/gpu_memory_total_gb": gpu_memory_total,
                "system/gpu_utilization_pct": (gpu_memory_used / gpu_memory_total) * 100
            })
        
        wandb.log(log_data)
        
        # Early stopping 체크 (모든 모델이 patience 초과 시)
        # if all(p >= max_patience for p in patience.values()) and epoch > EPOCHS // 2:
        #     print(f"⏸️ Early stopping at epoch {epoch+1} (모든 모델 patience 초과)")
        #     wandb.log({"early_stopping/epoch": epoch + 1})
        #     break
    
    # Fold 결과 저장
    for model_key in ACTIVE_MODELS:
        fold_result = {
            'fold': fold + 1,
            'best_val_f1': best_f1s[model_key],
            'model_name': MODEL_CONFIGS[model_key]['model_name'],
            'epochs_trained': epoch + 1,
        }
        all_fold_results[model_key].append(fold_result)
        all_fold_models[model_key].append(best_models[model_key])
    
    print(f"\n✅ Fold {fold + 1} 완료!")
    for model_key in ACTIVE_MODELS:
        print(f"  {model_key:10s}: 최고 F1 = {best_f1s[model_key]:.4f}")
    
    # Fold별 요약 로깅
    wandb.log({
        "fold_summary/completed": True,
        "fold_summary/epochs_trained": epoch + 1,
        **{f"fold_summary/{model_key}_best_f1": best_f1s[model_key] for model_key in ACTIVE_MODELS}
    })
    
    fold_run.finish()
    
    # 메모리 정리
    del models, optimizers, schedulers, trn_loaders, val_loaders
    torch.cuda.empty_cache()

In [ ]:
# K-Fold 결과 분석

print(f"\n{'='*60}")
print("🏁 K-FOLD 다중 모델 결과")
print(f"{'='*60}")

for model_key in ACTIVE_MODELS:
    val_f1_scores = [result['best_val_f1'] for result in all_fold_results[model_key]]
    mean_f1 = np.mean(val_f1_scores)
    std_f1 = np.std(val_f1_scores)
    
    print(f"\n🤖 {model_key.upper()} ({MODEL_CONFIGS[model_key]['model_name']}):")
    print(f"  📊 평균 CV F1: {mean_f1:.4f} ± {std_f1:.4f}")
    print(f"  🏆 최고 Fold: {max(val_f1_scores):.4f}")
    print(f"  📉 최악 Fold: {min(val_f1_scores):.4f}")
    print(f"  📏 성능 범위: {max(val_f1_scores) - min(val_f1_scores):.4f}")
    
    # 메인 run에 모델별 CV 결과 로깅
    try:
        main_run.log({
            f"cv_results/{model_key}_mean_f1": mean_f1,
            f"cv_results/{model_key}_std_f1": std_f1,
            f"cv_results/{model_key}_best_fold_f1": max(val_f1_scores),
            f"cv_results/{model_key}_worst_fold_f1": min(val_f1_scores),
            f"cv_results/{model_key}_f1_range": max(val_f1_scores) - min(val_f1_scores),
        })
    except:
        pass

print("\n📊 모델별 성능 비교:")
for fold in range(N_FOLDS):
    print(f"  Fold {fold+1}:", end="")
    for model_key in ACTIVE_MODELS:
        f1_score = all_fold_results[model_key][fold]['best_val_f1']
        print(f" {model_key}={f1_score:.4f}", end="")
    print()

In [ ]:
# 앙상블 모델 준비

ensemble_models = {model_key: [] for model_key in ACTIVE_MODELS}
print(f"\n🔧 앙상블 모델 준비 중...")

for model_key in ACTIVE_MODELS:
    for i, state_dict in enumerate(all_fold_models[model_key]):
        config_detail = MODEL_CONFIGS[model_key]
        fold_model = timm.create_model(config_detail['model_name'], pretrained=True, num_classes=17).to(device)
        fold_model.load_state_dict(state_dict)
        fold_model.eval()
        ensemble_models[model_key].append(fold_model)
        print(f"📁 {model_key} Fold {i+1} 모델 로드 완료")

print(f"\n🎯 앙상블 구성:")
for model_key in ACTIVE_MODELS:
    print(f"  {model_key}: {len(ensemble_models[model_key])}개 모델")

try:
    main_run.log({
        "ensemble/num_model_types": len(ACTIVE_MODELS),
        "ensemble/models_per_type": N_FOLDS,
        "ensemble/total_models": len(ACTIVE_MODELS) * N_FOLDS,
        "ensemble/architectures": [MODEL_CONFIGS[key]['architecture'] for key in ACTIVE_MODELS]
    })
except:
    pass

In [ ]:
# Temperature Scaling 클래스

class TemperatureScaling(nn.Module):
    def __init__(self, temperature=1.5):
        super().__init__()
        self.temperature = nn.Parameter(torch.ones(1) * temperature)
    
    def forward(self, logits):
        return logits / self.temperature

## TTA : Multi-Resoulition TTA

In [ ]:
# Multi-Resolution TTA Setup

print(f"\n🔄 다중 해상도 TTA (Test Time Augmentation) 설정...")

# 각 모델별 최적화된 TTA 변환 생성
multi_res_tta_transforms = {}

for model_key in ACTIVE_MODELS:
    img_size = MODEL_CONFIGS[model_key]['img_size']
    
    # 각 모델에 최적화된 TTA 변환들
    tta_transforms = [
        # 원본
        A.Compose([
            A.LongestMaxSize(max_size=img_size),
            A.PadIfNeeded(min_height=img_size, min_width=img_size, border_mode=0, value=0),
            A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
            ToTensorV2(),
        ]),
        # 90도 회전들
        A.Compose([
            A.LongestMaxSize(max_size=img_size),
            A.PadIfNeeded(min_height=img_size, min_width=img_size, border_mode=0, value=0),
            A.Rotate(limit=[90, 90], p=1.0),
            A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
            ToTensorV2(),
        ]),
        A.Compose([
            A.LongestMaxSize(max_size=img_size),
            A.PadIfNeeded(min_height=img_size, min_width=img_size, border_mode=0, value=0),
            A.Rotate(limit=[180, 180], p=1.0),
            A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
            ToTensorV2(),
        ]),
        A.Compose([
            A.LongestMaxSize(max_size=img_size),
            A.PadIfNeeded(min_height=img_size, min_width=img_size, border_mode=0, value=0),
            A.Rotate(limit=[-90, -90], p=1.0),
            A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
            ToTensorV2(),
        ]),
        # 밝기 조정
        A.Compose([
            A.LongestMaxSize(max_size=img_size),
            A.PadIfNeeded(min_height=img_size, min_width=img_size, border_mode=0, value=0),
            A.RandomBrightnessContrast(brightness_limit=[0.3, 0.3], contrast_limit=[0.3, 0.3], p=1.0),
            A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
            ToTensorV2(),
        ]),
    ]
    
    multi_res_tta_transforms[model_key] = tta_transforms
    print(f"✅ {model_key} ({img_size}x{img_size}) TTA 변환 {len(tta_transforms)}개 준비 완료")

try:
    main_run.log({
        "tta/multi_resolution": True,
        "tta/transforms_per_model": {
            model_key: len(transforms) for model_key, transforms in multi_res_tta_transforms.items()
        },
        "tta/image_sizes": {
            model_key: MODEL_CONFIGS[model_key]['img_size'] for model_key in ACTIVE_MODELS
        }
    })
except:
    pass

In [ ]:
# Multi-Resolution TTA Dataset 클래스

class MultiResTTAImageDataset(Dataset):
    def __init__(self, data, path, multi_transforms):
        """
        다중 해상도 TTA를 위한 Dataset
        multi_transforms: {model_key: [transform1, transform2, ...]} 형태
        """
        if isinstance(data, str):
            self.df = pd.read_csv(data).values
        else:
            self.df = data.values
        self.path = path
        self.multi_transforms = multi_transforms

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        name, target = self.df[idx]
        img = np.array(Image.open(os.path.join(self.path, name)))
        
        # 모델별, TTA 변형별로 이미지 생성
        augmented_images = {}
        for model_key, transforms in self.multi_transforms.items():
            model_images = []
            for transform in transforms:
                aug_img = transform(image=img)['image']
                model_images.append(aug_img)
            augmented_images[model_key] = model_images
        
        return augmented_images, target

# Multi-Resolution TTA Dataset 생성
multi_tta_dataset = MultiResTTAImageDataset(
    "/root/home/cv_contest/CV_data/sample_submission.csv",
    "/root/home/cv_contest/CV_data/test",
    multi_res_tta_transforms
)

# 각 모델별 최적 배치 크기로 DataLoader 생성
multi_tta_loaders = {}

for model_key in ACTIVE_MODELS:
    batch_size = min(MODEL_CONFIGS[model_key]['batch_size'], 16)  # TTA용으로 배치 크기 조정
    
    multi_tta_loaders[model_key] = DataLoader(
        multi_tta_dataset,
        batch_size=batch_size,
        shuffle=False,
        num_workers=num_workers,
        pin_memory=True
    )
    
    print(f"📊 {model_key} TTA DataLoader: 배치 크기 {batch_size}")

print(f"📊 Multi-Resolution TTA Dataset: {len(multi_tta_dataset)}개 테스트 샘플")

# =============================================================================
# 17. Multi-Resolution Ensemble + TTA Inference Function
# =============================================================================

def multi_resolution_ensemble_tta_inference(ensemble_models, loaders, multi_transforms, confidence_threshold=0.9):
    """
    다중 해상도 앙상블 + TTA 추론 with 상세 WandB 로깅
    """
    # Temperature scaling 초기화
    temp_scalings = {model_key: TemperatureScaling().to(device) for model_key in ACTIVE_MODELS}
    
    # 결과 저장용
    all_predictions = []
    all_confidences = []
    model_predictions = {model_key: [] for model_key in ACTIVE_MODELS}
    model_confidences = {model_key: [] for model_key in ACTIVE_MODELS}
    
    print(f"🚀 다중 해상도 앙상블 TTA 추론 시작...")
    print(f"📊 모델별 설정:")
    for model_key in ACTIVE_MODELS:
        config_detail = MODEL_CONFIGS[model_key]
        num_folds = len(ensemble_models[model_key])
        num_tta = len(multi_transforms[model_key])
        total_preds = num_folds * num_tta
        print(f"  🤖 {model_key}: {num_folds}개 모델 × {num_tta}개 TTA = {total_preds}개 예측")
    
    start_time = time.time()
    
    # 모델별 개별 추론 (메모리 효율성을 위해)
    for model_key in ACTIVE_MODELS:
        print(f"\n🔍 {model_key.upper()} 모델 추론 중...")
        
        config_detail = MODEL_CONFIGS[model_key]
        models = ensemble_models[model_key]
        loader = multi_tta_loaders[model_key]
        temp_scaling = temp_scalings[model_key]
        
        model_start_time = time.time()
        
        # 진행상황 로깅을 위한 테이블
        progress_table = wandb.Table(columns=["Batch", "Avg_Confidence", "Low_Conf_Count", "High_Conf_Count"])
        
        for batch_idx, batch_data in enumerate(tqdm(loader, desc=f"{model_key} TTA")):
            images_dict, _ = batch_data
            images_list = images_dict[model_key]  # 현재 모델의 TTA 이미지들
            batch_size = images_list[0].size(0)
            
            # 현재 모델의 앙상블 확률 초기화
            model_ensemble_probs = torch.zeros(batch_size, 17).to(device)
            
            # 각 fold 모델별 예측
            for fold_model in models:
                fold_model.eval()
                with torch.no_grad():
                    # 각 TTA 변형별 예측
                    for tta_images in images_list:
                        tta_images = tta_images.to(device)
                        preds = fold_model(tta_images)
                        
                        # Temperature scaling 적용
                        preds = temp_scaling(preds)
                        probs = torch.softmax(preds, dim=1)
                        
                        # 앙상블 확률에 누적
                        model_ensemble_probs += probs / (len(models) * len(images_list))
            
            # 현재 모델의 신뢰도 계산
            max_probs = torch.max(model_ensemble_probs, dim=1)[0]
            batch_confidences = max_probs.cpu().numpy()
            
            final_preds = torch.argmax(model_ensemble_probs, dim=1)
            
            # 모델별 결과 저장
            model_predictions[model_key].extend(final_preds.cpu().numpy())
            model_confidences[model_key].extend(batch_confidences)
            
            # 배치별 신뢰도 분석
            high_conf_count = np.sum(batch_confidences >= confidence_threshold)
            low_conf_count = batch_size - high_conf_count
            avg_confidence = np.mean(batch_confidences)
            
            progress_table.add_data(batch_idx, avg_confidence, low_conf_count, high_conf_count)
            
            # 배치별 상세 로깅 (20배치마다)
            if batch_idx % 20 == 0:
                elapsed_time = time.time() - model_start_time
                try:
                    main_run.log({
                        f"tta_progress/{model_key}_batch": batch_idx,
                        f"tta_progress/{model_key}_avg_confidence": avg_confidence,
                        f"tta_progress/{model_key}_high_confidence_ratio": high_conf_count / batch_size,
                        f"tta_progress/{model_key}_elapsed_time_min": elapsed_time / 60,
                    })
                except:
                    pass
        
        model_time = time.time() - model_start_time
        model_avg_confidence = np.mean(model_confidences[model_key])
        
        # 모델별 최종 결과 로깅
        try:
            main_run.log({
                f"tta_results/{model_key}_time_min": model_time / 60,
                f"tta_results/{model_key}_avg_confidence": model_avg_confidence,
                f"tta_results/{model_key}_confidence_std": np.std(model_confidences[model_key]),
                f"tta_results/{model_key}_high_conf_samples": np.sum(np.array(model_confidences[model_key]) >= confidence_threshold),
                f"tta_results/{model_key}_progress_table": progress_table
            })
        except:
            pass
        
        print(f"✅ {model_key} 완료! 소요시간: {model_time/60:.1f}분, 평균 신뢰도: {model_avg_confidence:.4f}")
        
        # 메모리 정리
        torch.cuda.empty_cache()

    # =============================================================================
    # 모델 간 앙상블 (단순 평균으로 시작)
    # =============================================================================
    
    print(f"\n🔗 모델 간 앙상블 수행 중...")
    
    for i in range(len(multi_tta_dataset)):
        # 각 모델의 예측을 단순 평균
        ensemble_pred_probs = []
        for model_key in ACTIVE_MODELS:
            # 예측을 one-hot으로 변환 후 평균
            pred = model_predictions[model_key][i]
            confidence = model_confidences[model_key][i]
            
            # 신뢰도를 고려한 soft voting
            prob_vector = np.zeros(17)
            prob_vector[pred] = confidence
            ensemble_pred_probs.append(prob_vector)
        
        # 단순 평균
        final_prob = np.mean(ensemble_pred_probs, axis=0)
        final_pred = np.argmax(final_prob)
        final_confidence = np.max(final_prob)
        
        all_predictions.append(final_pred)
        all_confidences.append(final_confidence)
    
    total_time = time.time() - start_time
    
    # =============================================================================
    # 최종 앙상블 결과 분석 및 로깅
    # =============================================================================
    
    final_avg_confidence = np.mean(all_confidences)
    confidence_std = np.std(all_confidences)
    high_conf_samples = np.sum(np.array(all_confidences) >= confidence_threshold)
    
    print(f"\n✅ 다중 해상도 앙상블 TTA 완료!")
    print(f"⏰ 총 소요시간: {total_time/60:.1f}분")
    print(f"🎯 최종 평균 신뢰도: {final_avg_confidence:.4f} ± {confidence_std:.4f}")
    print(f"🏆 고신뢰도 샘플: {high_conf_samples}/{len(all_predictions)} ({high_conf_samples/len(all_predictions)*100:.1f}%)")
    
    # 모델별 성능 분석
    print(f"\n📊 모델별 신뢰도 분석:")
    for model_key in ACTIVE_MODELS:
        model_avg_conf = np.mean(model_confidences[model_key])
        model_high_conf = np.sum(np.array(model_confidences[model_key]) >= confidence_threshold)
        print(f"  {model_key:12s}: 평균 {model_avg_conf:.4f}, 고신뢰도 {model_high_conf}/{len(model_confidences[model_key])} ({model_high_conf/len(model_confidences[model_key])*100:.1f}%)")
    
    # 최종 결과 로깅
    try:
        main_run.log({
            "final_ensemble/total_time_min": total_time / 60,
            "final_ensemble/samples_per_second": len(all_predictions) / total_time,
            "final_ensemble/final_avg_confidence": final_avg_confidence,
            "final_ensemble/confidence_std": confidence_std,
            "final_ensemble/high_confidence_samples": high_conf_samples,
            "final_ensemble/high_confidence_ratio": high_conf_samples / len(all_predictions),
            "final_ensemble/total_predictions": len(all_predictions),
            "final_ensemble/confidence_histogram": wandb.Histogram(all_confidences),
            "final_ensemble/ensemble_method": "simple_average",
            "final_ensemble/num_model_types": len(ACTIVE_MODELS),
        })
        
        # 모델별 개별 성능도 로깅
        for model_key in ACTIVE_MODELS:
            main_run.log({
                f"model_performance/{model_key}_avg_confidence": np.mean(model_confidences[model_key]),
                f"model_performance/{model_key}_confidence_std": np.std(model_confidences[model_key]),
                f"model_performance/{model_key}_high_conf_ratio": np.sum(np.array(model_confidences[model_key]) >= confidence_threshold) / len(model_confidences[model_key])
            })
        
        print("📊 최종 앙상블 결과 WandB 로깅 완료!")
    except Exception as e:
        print(f"⚠️ WandB 로깅 중 에러: {e}")
    
    return all_predictions, all_confidences, model_predictions, model_confidences


In [ ]:
# Multi-Resolution Ensemble TTA 실행

print(f"\n{'='*60}")
print("🎯 최종 추론 - 다중 해상도 앙상블 + TTA")
print(f"{'='*60}")

multi_tta_predictions, multi_confidences, individual_model_preds, individual_model_confs = multi_resolution_ensemble_tta_inference(
    ensemble_models=ensemble_models,
    loaders=multi_tta_loaders,
    multi_transforms=multi_res_tta_transforms,
    confidence_threshold=0.9
)

In [ ]:
# Multi-Resolution Results and Submission

print(f"\n📊 다중 해상도 앙상블 결과 정리 중...")

# 최종 submission 파일 생성
multi_pred_df = pd.DataFrame(multi_tta_dataset.df, columns=['ID', 'target'])
multi_pred_df['target'] = multi_tta_predictions

# ID 순서 확인
sample_submission_df = pd.read_csv("/root/home/cv_contest/CV_data/sample_submission.csv")
assert (sample_submission_df['ID'] == multi_pred_df['ID']).all(), "❌ ID 순서 불일치!"

# 예측 분포 분석
pred_distribution = multi_pred_df['target'].value_counts().sort_index()
pred_table = wandb.Table(columns=["Class", "Count", "Percentage", "Swin_B_Count", "ConvNeXt_Count"])

print(f"\n📊 최종 예측 결과 분포 (다중 해상도 앙상블):")
for class_id in range(17):
    final_count = pred_distribution.get(class_id, 0)
    final_percentage = final_count / len(multi_pred_df) * 100
    
    # 각 모델별 예측 분포도 확인
    swin_count = np.sum(np.array(individual_model_preds['swin_b']) == class_id) if 'swin_b' in individual_model_preds else 0
    convnext_count = np.sum(np.array(individual_model_preds['convnext']) == class_id) if 'convnext' in individual_model_preds else 0
    
    pred_table.add_data(class_id, final_count, final_percentage, swin_count, convnext_count)
    print(f"Class {class_id:2d}: {final_count:4d} ({final_percentage:5.1f}%) [Swin-B: {swin_count}, ConvNeXt: {convnext_count}]")

# 신뢰도 상세 분석
confidence_analysis = {}
confidence_bins = [0.5, 0.7, 0.8, 0.9, 0.95, 1.0]
for i, threshold in enumerate(confidence_bins):
    if i == 0:
        count = np.sum(np.array(multi_confidences) >= threshold)
    else:
        prev_threshold = confidence_bins[i-1]
        count = np.sum((np.array(multi_confidences) >= prev_threshold) & (np.array(multi_confidences) < threshold))
    confidence_analysis[f"conf_{threshold}"] = count

# 최종 결과 로깅
try:
    main_run.log({
        "final_results/multi_resolution": True,
        "final_results/total_predictions": len(multi_tta_predictions),
        "final_results/unique_classes_predicted": len(np.unique(multi_tta_predictions)),
        "final_results/prediction_distribution_table": pred_table,
        "final_results/avg_confidence": np.mean(multi_confidences),
        "final_results/median_confidence": np.median(multi_confidences),
        "final_results/min_confidence": np.min(multi_confidences),
        "final_results/max_confidence": np.max(multi_confidences),
        "final_results/confidence_distribution": wandb.Histogram(multi_confidences),
        "final_results/ensemble_type": "multi_resolution_simple_average",
        **confidence_analysis
    })
    
    # 예측 분포 바차트
    pred_dist_data = [[f"Class_{i}", pred_distribution.get(i, 0)] for i in range(17)]
    main_run.log({
        "final_results/prediction_distribution_chart": wandb.plot.bar(
            wandb.Table(data=pred_dist_data, columns=["Class", "Count"]),
            "Class", "Count", 
            title="Multi-Resolution Ensemble Prediction Distribution"
        )
    })
    
    print("📊 다중 해상도 앙상블 결과 로깅 완료!")
except Exception as e:
    print(f"⚠️ WandB 로깅 중 에러: {e}")

# 결과 저장
output_path = f"/root/home/cv_contest/results/multiRes_ensemble_{datetime.now().strftime('%Y%m%d_%H%M')}.csv"
multi_pred_df.to_csv(output_path, index=False)

# 결과 파일을 WandB 아티팩트로 저장
try:
    artifact = wandb.Artifact(
        name="multi_resolution_ensemble_predictions",
        type="predictions",
        description=f"Multi-Resolution Ensemble (Swin-B + ConvNeXt) predictions with {N_FOLDS}-fold CV + TTA"
    )
    artifact.add_file(output_path)
    main_run.log_artifact(artifact)
    print("📦 다중 해상도 앙상블 아티팩트 저장 완료!")
except Exception as e:
    print(f"⚠️ 아티팩트 저장 중 에러: {e}")

print(f"\n✅ 다중 해상도 앙상블 결과 저장 완료!")
print(f"📁 파일 위치: {output_path}")
print(f"📈 총 예측 수: {len(multi_tta_predictions)}")
print(f"🤖 사용된 모델: {', '.join(ACTIVE_MODELS)}")
print(f"🔥 사용된 해상도: {', '.join([f'{k}={v}px' for k, v in img_sizes.items()])}")

In [ ]:
# 모델별 개별 결과 파일도 저장

for model_key in ACTIVE_MODELS:
    individual_pred_df = pd.DataFrame(multi_tta_dataset.df, columns=['ID', 'target'])
    individual_pred_df['target'] = individual_model_preds[model_key]
    
    individual_output_path = f"/root/home/cv_contest/results/{model_key}_individual_{datetime.now().strftime('%Y%m%d_%H%M')}.csv"
    individual_pred_df.to_csv(individual_output_path, index=False)
    
    # 개별 모델 결과 요약
    individual_dist = individual_pred_df['target'].value_counts().sort_index()
    individual_avg_conf = np.mean(individual_model_confs[model_key])
    
    print(f"📁 {model_key} 개별 결과: {individual_output_path}")
    print(f"   평균 신뢰도: {individual_avg_conf:.4f}")
    
    # WandB에 개별 모델 아티팩트 저장
    try:
        individual_artifact = wandb.Artifact(
            name=f"{model_key}_individual_predictions",
            type="individual_predictions",
            description=f"{model_key} individual predictions with {N_FOLDS}-fold CV + TTA"
        )
        individual_artifact.add_file(individual_output_path)
        main_run.log_artifact(individual_artifact)
    except Exception as e:
        print(f"⚠️ {model_key} 개별 아티팩트 저장 중 에러: {e}")


In [ ]:
# 최종 실험 요약 및 정리

print(f"\n{'='*60}")
print("🏁 다중 해상도 앙상블 실험 완료!")
print(f"{'='*60}")

# 전체 실험 요약
total_models_trained = len(ACTIVE_MODELS) * N_FOLDS
total_tta_transforms = sum(len(transforms) for transforms in multi_res_tta_transforms.values())
total_predictions_per_sample = total_models_trained * (total_tta_transforms // len(ACTIVE_MODELS))

print(f"📊 실험 요약:")
print(f"   🤖 사용된 아키텍처: {len(ACTIVE_MODELS)}개 ({', '.join(ACTIVE_MODELS)})")
print(f"   📁 총 학습된 모델: {total_models_trained}개 ({N_FOLDS}-Fold CV)")
print(f"   🔄 TTA 변형: 모델당 평균 {total_tta_transforms // len(ACTIVE_MODELS)}개")
print(f"   🎯 샘플당 예측 수: {total_predictions_per_sample}개")
print(f"   📈 테스트 샘플: {len(multi_tta_predictions)}개")

# CV 결과 요약
print(f"\n📊 Cross-Validation 결과:")
cv_summary_table = wandb.Table(columns=["Model", "Mean_F1", "Std_F1", "Best_Fold", "Worst_Fold", "Architecture"])

for model_key in ACTIVE_MODELS:
    val_f1_scores = [result['best_val_f1'] for result in all_fold_results[model_key]]
    mean_f1 = np.mean(val_f1_scores)
    std_f1 = np.std(val_f1_scores)
    best_f1 = max(val_f1_scores)
    worst_f1 = min(val_f1_scores)
    architecture = MODEL_CONFIGS[model_key]['architecture']
    
    cv_summary_table.add_data(model_key, mean_f1, std_f1, best_f1, worst_f1, architecture)
    print(f"   {model_key:12s}: {mean_f1:.4f} ± {std_f1:.4f} (범위: {worst_f1:.4f} ~ {best_f1:.4f})")

# 최종 성능 지표
final_metrics = {
    "experiment_type": "multi_resolution_ensemble",
    "total_experiment_time_hours": (time.time() - start_time) / 3600,
    "models_trained": total_models_trained,
    "cv_folds": N_FOLDS,
    "tta_transforms_total": total_tta_transforms,
    "test_samples": len(multi_tta_predictions),
    "unique_classes_in_prediction": len(np.unique(multi_tta_predictions)),
    "final_ensemble_confidence": np.mean(multi_confidences),
    "high_confidence_predictions_ratio": np.sum(np.array(multi_confidences) >= 0.9) / len(multi_confidences),
    "cv_summary_table": cv_summary_table
}

# 최종 WandB 로깅
try:
    main_run.log(final_metrics)
    
    # 실험 요약 텍스트
    experiment_summary = f"""
# Multi-Resolution Ensemble Experiment Summary

## Models Used
- **Swin Transformer Base**: {MODEL_CONFIGS['swin_b']['img_size']}px resolution
- **ConvNeXt Base**: {MODEL_CONFIGS['convnext']['img_size']}px resolution

## Training Strategy
- **Cross-Validation**: {N_FOLDS}-Fold Stratified
- **Total Models Trained**: {total_models_trained}
- **Test Time Augmentation**: {total_tta_transforms // len(ACTIVE_MODELS)} transforms per model

## Performance
- **Final Ensemble Confidence**: {np.mean(multi_confidences):.4f}
- **High Confidence Predictions**: {np.sum(np.array(multi_confidences) >= 0.9)}/{len(multi_confidences)} ({np.sum(np.array(multi_confidences) >= 0.9)/len(multi_confidences)*100:.1f}%)

## CV Results
{chr(10).join([f"- **{model_key}**: {np.mean([r['best_val_f1'] for r in all_fold_results[model_key]]):.4f} ± {np.std([r['best_val_f1'] for r in all_fold_results[model_key]]):.4f}" for model_key in ACTIVE_MODELS])}
    """
    
    main_run.summary.update({
        "experiment_summary": experiment_summary,
        "best_cv_f1": max([np.mean([r['best_val_f1'] for r in all_fold_results[model_key]]) for model_key in ACTIVE_MODELS]),
        "total_runtime_hours": (time.time() - start_time) / 3600,
        "final_prediction_file": output_path,
        "ensemble_method": "multi_resolution_average"
    })
    
    print("📊 최종 실험 요약 WandB 로깅 완료!")
    
except Exception as e:
    print(f"⚠️ 최종 로깅 중 에러: {e}")

In [ ]:
# 메모리 정리 및 실험 종료

# GPU 메모리 정리
if torch.cuda.is_available():
    for model_key in ensemble_models:
        for model in ensemble_models[model_key]:
            del model
    del ensemble_models
    torch.cuda.empty_cache()
    
    print(f"🧹 GPU 메모리 정리 완료")

# WandB 실험 종료
main_run.finish()

print(f"\n{'='*60}")
print("🎉 다중 해상도 앙상블 실험 완료!")
print(f"{'='*60}")
print(f"📁 최종 결과 파일: {output_path}")
print(f"📊 WandB 프로젝트: {PROJECT_NAME}")
print(f"⏰ 총 실행 시간: {(time.time() - start_time) / 3600:.1f}시간")
print(f"🏆 최종 앙상블 평균 신뢰도: {np.mean(multi_confidences):.4f}")
print(f"🎯 고신뢰도 예측 비율: {np.sum(np.array(multi_confidences) >= 0.9)/len(multi_confidences)*100:.1f}%")

# 최종 성공 메시지
print(f"\n✨ 실험이 성공적으로 완료되었습니다!")
print(f"📈 결과를 WandB 대시보드에서 확인하세요.")